In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error
pd.options.display.max_columns = 100
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
players = pd.read_csv('MLB_players19.csv')

In [3]:
df = pd.read_parquet('sc19.parquet')

In [4]:
df = df.loc[df.game_type == 'R'].copy()

MemoryError: Unable to allocate 5.45 MiB for an array with shape (5713229,) and data type bool

In [4]:
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,SL,2019-10-30,87.9,-2.65,5.50,"Hudson, Daniel",488726,543339,strikeout,swinging_strike,<NA>,<NA>,<NA>,<NA>,14,Michael Brantley strikes out swinging.,W,L,R,HOU,WSH,S,2,None,3,2,2019,0.02,0.21,0.88,1.03,<NA>,<NA>,<NA>,2,9,Bot,NaN,NaN,<NA>,<NA>,543228,<NA>,None,8.472927,-127.797998,-5.181622,-1.384471,23.932114,-29.091156,3.35,1.40,<NA>,NaN,<NA>,87.8,2461,6.1,599377,543339,543228,475582,452678,543685,607208,665742,645302,594809,54.42,NaN,NaN,0.0,1,0,0,<NA>,79,7,Slider,2,6,2,6,6,2,2,6,Infield shift,Standard,<NA>,NaN,NaN
1,FF,2019-10-30,95.9,-2.77,5.52,"Hudson, Daniel",488726,543339,None,foul,<NA>,<NA>,<NA>,<NA>,7,Michael Brantley strikes out swinging.,W,L,R,HOU,WSH,S,<NA>,None,3,2,2019,-0.57,1.52,-0.47,1.92,<NA>,<NA>,<NA>,2,9,Bot,NaN,NaN,<NA>,<NA>,543228,<NA>,None,7.236909,-139.212420,-7.101361,-8.940442,32.683954,-11.563514,3.35,1.56,276,87.9,33,94.4,2572,5.7,599377,543339,543228,475582,452678,543685,607208,665742,645302,594809,54.83,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,79,6,4-Seam Fastball,2,6,2,6,6,2,2,6,Infield shift,Standard,<NA>,NaN,NaN
2,FF,2019-10-30,96.5,-2.68,5.42,"Hudson, Daniel",488726,543339,None,ball,<NA>,<NA>,<NA>,<NA>,14,Michael Brantley strikes out swinging.,W,L,R,HOU,WSH,B,<NA>,None,2,2,2019,-0.66,1.40,1.68,1.35,<NA>,<NA>,<NA>,2,9,Bot,NaN,NaN,<NA>,<NA>,543228,<NA>,None,12.923798,-139.703293,-8.230555,-11.485524,32.459653,-12.442450,3.53,1.63,<NA>,NaN,<NA>,95.3,2637,5.9,599377,543339,543228,475582,452678,543685,607208,665742,645302,594809,54.55,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,79,5,4-Seam Fastball,2,6,2,6,6,2,2,6,Infield shift,Standard,<NA>,NaN,NaN
3,FF,2019-10-30,96.0,-2.65,5.55,"Hudson, Daniel",488726,543339,None,foul,<NA>,<NA>,<NA>,<NA>,9,Michael Brantley strikes out swinging.,W,L,R,HOU,WSH,S,<NA>,None,2,1,2019,-0.81,1.50,0.75,2.05,<NA>,<NA>,<NA>,2,9,Bot,NaN,NaN,<NA>,<NA>,543228,<NA>,None,10.686962,-139.133193,-6.897620,-12.790017,31.422309,-11.667275,3.35,1.56,380,105.3,25,94.9,2598,5.9,599377,543339,543228,475582,452678,543685,607208,665742,645302,594809,54.60,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,79,4,4-Seam Fastball,2,6,2,6,6,2,2,6,Infield shift,Standard,<NA>,NaN,NaN
4,SL,2019-10-30,86.7,-2.73,5.59,"Hudson, Daniel",488726,543339,None,ball,<NA>,<NA>,<NA>,<NA>,14,Michael Brantley strikes out swinging.,W,L,R,HOU,WSH,B,<NA>,None,1,1,2019,-0.05,0.47,1.27,2.17,<NA>,<NA>,<NA>,2,9,Bot,NaN,NaN,<NA>,<NA>,543228,<NA>,None,9.633513,-126.044028,-2.982919,-2.223969,21.921075,-26.826688,3.59,1.63,<NA>,NaN,<NA>,87.0,2598,6.2,599377,543339,543228,475582,452678,543685,607208,665742,645302,594809,54.28,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,79,3,Slider,2,6,2,6,6,2,2,6,Infield shift,Standard,<NA>,NaN,NaN


In [49]:
# number of ABs per game by each pitcher
bfpg = df.groupby(['player_name','pitcher','game_year','game_pk']).at_bat_number.agg(lambda x : x.nunique()).reset_index()
bfpg.head()

# number of ABs per year by each pitcher
bf = bfpg.groupby(['player_name','pitcher','game_year']).at_bat_number.sum().reset_index()
bf200 = bf[bf.at_bat_number >= 200]

In [63]:
# at bats resulting in strikeouts
dfk = df[df.events.str.contains('strikeout', na = False)]
# strikeouts by pitcher in given year
totalk = dfk.groupby(['player_name','pitcher','game_year']).events.agg(lambda x : x.count().sum()).reset_index()
totalk = totalk.rename(columns = {'events':'strikeouts'})

# at bats resulting in walks or intent walks
dfbb = df[df.events.str.contains('walk', na = False)]
# walks by pitcher in given year
totalbb = dfbb.groupby(['player_name','pitcher','game_year']).events.agg(lambda x : x.count().sum()).reset_index()
totalbb = totalbb.rename(columns = {'events':'walks'})

In [69]:
bbk200 = bf200.merge(totalk, on = ['player_name','pitcher','game_year']).merge(totalbb, on = ['player_name','pitcher','game_year'])

In [73]:
pitcher_list = bbk200.player_name.value_counts()
pitcher_list = pitcher_list[pitcher_list > 1].index.unique().to_list()
pitchers200 = bbk200[bbk200.player_name.isin(pitcher_list)]

In [76]:
pitchers200['bbrate'] = pitchers200.walks/pitchers200.at_bat_number
pitchers200['krate'] = pitchers200.strikeouts/(pitchers200.at_bat_number - pitchers200.walks)
pitchers200['ydiff'] = pitchers200.groupby('player_name')['game_year'].diff(1).fillna(0)
pitchers200['bbrate_prev'] = pitchers200.bbrate.shift()
pitchers200['krate_prev'] = pitchers200.krate.shift()
pitchersprev = pitchers200[pitchers200.ydiff == 1]
pitchersprev['yprev'] = pitchersprev.game_year - 1

In [91]:
# 1.a.i.
res = smf.ols('bbrate ~ bbrate_prev', pitchersprev[pitchersprev.game_year < 2019]).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 bbrate   R-squared:                       0.267
Model:                            OLS   Adj. R-squared:                  0.266
Method:                 Least Squares   F-statistic:                     524.3
Date:                Wed, 07 Apr 2021   Prob (F-statistic):           3.16e-99
Time:                        16:05:30   Log-Likelihood:                 3620.5
No. Observations:                1444   AIC:                            -7237.
Df Residuals:                    1442   BIC:                            -7226.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0369      0.002     20.276      0.000       0.033       0.040
bbrate_prev     0.5204      0.023     22.898      0.000       0.476       0.565
==============================================================================
Omnibus:                       82.684   Durbin-Watson:                   2.107
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              114.381
Skew:                           0.506   Prob(JB):                     1.45e-25
Kurtosis:                       3.937   Cond. No.                         44.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# 1.a.ii.
y_true = pitchersprev[pitchersprev.game_year == 2019].bbrate
X = pitchersprev[pitchersprev.game_year == 2019].drop('bbrate', axis = 1)

In [101]:
ypred = res.predict(X)
print(f'Mean squared error for bbrate 2019 is: {mean_squared_error(y_true, ypred)}')

Mean squared error for bbrate 2019 is: 0.0004498653000155517


In [109]:
# 1.b.i.
res2 = smf.ols('krate ~ krate_prev', pitchersprev[pitchersprev.game_year < 2019]).fit()
res2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  krate   R-squared:                       0.555
Model:                            OLS   Adj. R-squared:                  0.555
Method:                 Least Squares   F-statistic:                     1800.
Date:                Wed, 07 Apr 2021   Prob (F-statistic):          6.46e-256
Time:                        16:13:57   Log-Likelihood:                 2498.0
No. Observations:                1444   AIC:                            -4992.
Df Residuals:                    1442   BIC:                            -4981.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0592      0.004     13.768      0.000       0.051       0.068
krate_prev     0.7482      0.018     42.421      0.000       0.714       0.783
==============================================================================
Omnibus:                       89.323   Durbin-Watson:                   2.219
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              115.323
Skew:                           0.564   Prob(JB):                     9.08e-26
Kurtosis:                       3.801   Cond. No.                         16.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [110]:
# 1.b.ii.
y_true2 = pitchersprev[pitchersprev.game_year == 2019].krate
X2 = pitchersprev[pitchersprev.game_year == 2019].drop('krate', axis = 1)

In [111]:
ypred2 = res2.predict(X)
print(f'Mean squared error for krate 2019 is: {mean_squared_error(y_true2, ypred2)}')

Mean squared error for krate 2019 is: 0.0019347005945504721


In [4]:
from sklearn.metrics import brier_score_loss
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold
from sklearn.pipeline import Pipeline

In [5]:
filt = ['ball', 'blocked_ball', 'called_strike']
dfcalled = df[df['description'].isin(filt)]

In [12]:
dfcalled['target'] = np.where(dfcalled.description == 'called_strike', 1, 0)
features = ['game_year','pitch_type', 'pitch_name', 'release_pos_x', 'release_pos_z', 'zone', 'stand', 'p_throws', 'balls', 
            'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'vx0', 'vy0', 'sz_top', 'sz_bot', 'vz0', 'ax', 'ay', 'az', 
            'outs_when_up', 'on_1b', 'on_2b', 'on_3b', 'release_speed', 'release_spin_rate', 'release_extension', 'target']
dfmodel = dfcalled[features]

MemoryError: Unable to allocate 374. MiB for an array with shape (16, 3061621) and data type float64

In [15]:
x_train = dfmodel[dfmodel.game_year.between(2012,2017)].drop(['target','description'], axis = 1)
y_train = dfmodel[dfmodel.game_year.between(2012,2017)].target
x_test = dfmodel[dfmodel.game_year == 2018].drop(['target','description'], axis = 1)
y_test = dfmodel[dfmodel.game_year == 2018].target

MemoryError: Unable to allocate 279. MiB for an array with shape (16, 2285504) and data type float64

In [14]:
x_train.head()

,game_year,pitch_type,pitch_name,release_pos_x,release_pos_z,zone,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,sz_top,sz_bot,vz0,ax,ay,az,outs_when_up,on_1b,on_2b,on_3b,release_speed,release_spin_rate,release_extension,description
1474782,2017,FF,4-Seam Fastball,-1.92,5.87,12,R,R,1,1,-0.99,1.33,1.13,2.67,10.217639,-139.985298,3.48,1.65,-5.846387,-15.151904,32.028166,-13.810093,1,<NA>,<NA>,<NA>,96.7,2208,5.9,ball
1474784,2017,FF,4-Seam Fastball,-1.85,5.90,12,R,R,0,0,-1.01,1.17,1.01,2.65,9.644597,-139.249926,3.47,1.67,-5.516481,-15.055670,30.539018,-16.212462,1,<NA>,<NA>,<NA>,96.1,2188,5.8,ball
1474787,2017,FF,4-Seam Fastball,-2.02,5.86,8,L,R,0,0,-1.03,0.79,0.04,1.94,7.644091,-139.253172,3.53,1.72,-6.437605,-14.895041,31.458173,-20.841681,0,<NA>,<NA>,<NA>,96.1,2258,5.9,called_strike
1474791,2017,CH,Changeup,2.52,5.40,14,R,L,2,2,1.57,0.91,0.90,1.29,-6.814785,-121.824507,3.56,1.57,-4.868067,16.992177,24.177310,-22.337244,1,<NA>,<NA>,<NA>,84.0,1759,6.4,called_strike
1474794,2017,CH,Changeup,2.41,5.59,14,R,L,1,0,1.69,1.24,0.53,1.27,-7.527927,-121.013552,3.56,1.63,-5.798540,17.882932,24.074320,-19.261530,1,<NA>,<NA>,<NA>,83.5,1607,6.0,ball


In [5]:
vals = list(df.description.unique())
s = ['swinging_strike', 'swinging_strike_blocked']
vals = [v for v in vals if v not in s]

In [12]:
dict.fromkeys([v for v in vals], 0)
dict.fromkeys([i for i in s], 1)

{'foul': 0,
 'ball': 0,
 'called_strike': 0,
 'hit_into_play': 0,
 'blocked_ball': 0,
 'hit_by_pitch': 0,
 'foul_bunt': 0,
 'foul_tip': 0,
 'missed_bunt': 0,
 'pitchout': 0,
 'bunt_foul_tip': 0,
 'foul_pitchout': 0,
 'swinging_pitchout': 0,
 'intent_ball': 0}

In [18]:
test = df.head(10)
test2 = test.replace(dict.fromkeys([v for v in vals], 0)).replace(dict.fromkeys([i for i in s], 1))

In [20]:
#dfstrike = df.replace(dict.fromkeys([v for v in vals], 0)).replace(dict.fromkeys([i for i in s], 1))
#dfstrike.head()
df['target'] = np.where(df.description.isin(s), 1, 0)


In [24]:
#dfstrike = df[df.description.str.contains('swinging_s', na = False)]
vals = 
vals = ['swinging_strike, swinging_strike_blocked']
dfstrike = pd.DataFrame.where(cond = df.description.isin(vals), self = 1, other = 0)
dfstrike.head()

AttributeError: 'int' object has no attribute '_where'

In [20]:
df.description.values

array(['swinging_strike', 'foul', 'ball', ..., 'ball', 'hit_into_play',
       'called_strike'], dtype=object)

In [15]:
features = ['game_year','pitch_type', 'pitch_name', 'release_pos_x', 'release_pos_z', 'zone', 'stand', 'p_throws', 'balls', 'strikes', 
            'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'vx0', 'vy0', 'sz_top', 'sz_bot', 'vz0', 'ax', 'ay', 'az', 'outs_when_up', 
            'on_1b', 'on_2b', 'on_3b', 'release_speed', 'release_spin_rate', 'release_extension', 'description']
dfmodel = dfstrike[features]

In [16]:
dfmodel.head()

,game_year,pitch_type,pitch_name,release_pos_x,release_pos_z,zone,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,sz_top,sz_bot,vz0,ax,ay,az,outs_when_up,on_1b,on_2b,on_3b,release_speed,release_spin_rate,release_extension,description
0,2019,SL,Slider,-2.65,5.50,14,L,R,3,2,0.02,0.21,0.88,1.03,8.472927,-127.797998,3.35,1.40,-5.181622,-1.384471,23.932114,-29.091156,2,<NA>,<NA>,<NA>,87.9,2461,6.1,swinging_strike
7,2019,FF,4-Seam Fastball,-2.68,5.61,5,R,R,0,2,-0.69,1.47,0.23,2.69,9.126961,-138.980699,3.35,1.56,-5.288691,-10.809091,30.056700,-12.493125,1,<NA>,<NA>,<NA>,95.7,2541,5.8,swinging_strike
31,2019,SL,Slider,-2.52,3.07,12,R,R,0,0,1.64,0.84,1.17,2.34,4.979675,-114.980156,3.25,1.16,3.680321,13.504793,22.463635,-25.693025,1,543228,<NA>,<NA>,79.1,2521,6.3,swinging_strike
37,2019,SL,Slider,2.27,6.30,7,R,L,3,2,-0.42,-0.03,-0.63,2.10,-5.836947,-120.563835,3.43,1.63,-3.071787,-3.004945,22.687948,-32.074369,2,<NA>,<NA>,<NA>,82.8,2357,6.6,swinging_strike
50,2019,FT,2-Seam Fastball,2.38,6.35,14,R,L,1,1,1.45,0.86,1.00,2.37,-6.832388,-136.868908,3.47,1.64,-6.511650,19.794816,31.807211,-20.120004,0,<NA>,<NA>,<NA>,94.2,2317,6.5,swinging_strike


In [ ]:
x_train = dfmodel[dfmodel.game_year.between(2012,2017)].drop('description', axis = 1)
y_train = dfmodel[dfmodel.game_year.between(2012,2017)].description
x_test = dfmodel[dfmodel.game_year == 2018].drop('description', axis = 1)
y_test = datamaster[datamaster['Y'] == 18]['HomeWin']

In [ ]:
kfolds = KFold(n_splits = 4)

#Create a set of steps. All but the last step is a transformer (something that processes data). 
#Build a list of steps, where the first is StandardScaler and the last is RandomForest
steps = [('scaler', StandardScaler()),
         ('lr', LogisticRegression(solver = 'liblinear'))]

#Now set up the pipeline
pipeline = Pipeline(steps)

#Now set up the parameter grid
parameters_scaler = dict(lr__C = [10**i for i in range(-3, 3)],
                         lr__penalty = ['l1', 'l2'],
                         lr__max_iter = [100, 1000, 10000])

#Now run a grid search
lr_grid_search_scaler = GridSearchCV(pipeline, param_grid = parameters_scaler, cv = kfolds, scoring = 'neg_brier_score')

In [ ]:
#Now fit model to training data
lr_grid_search_scaler.fit(x_train, y_train)

proba = lr_grid_search_scaler.predict_proba(x_test)
brierscore = brier_score_loss(y_test, proba[:,1])

In [10]:
dfmodel.head()

,pitch_type,pitch_name,release_pos_x,release_pos_z,zone,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,sz_top,sz_bot,vz0,ax,ay,az,outs_when_up,on_1b,on_2b,on_3b,release_speed,release_spin_rate,release_extension,description
0,SL,Slider,-2.65,5.50,14,L,R,3,2,0.02,0.21,0.88,1.03,8.472927,-127.797998,3.35,1.40,-5.181622,-1.384471,23.932114,-29.091156,2,<NA>,<NA>,<NA>,87.9,2461,6.1,swinging_strike
7,FF,4-Seam Fastball,-2.68,5.61,5,R,R,0,2,-0.69,1.47,0.23,2.69,9.126961,-138.980699,3.35,1.56,-5.288691,-10.809091,30.056700,-12.493125,1,<NA>,<NA>,<NA>,95.7,2541,5.8,swinging_strike
31,SL,Slider,-2.52,3.07,12,R,R,0,0,1.64,0.84,1.17,2.34,4.979675,-114.980156,3.25,1.16,3.680321,13.504793,22.463635,-25.693025,1,543228,<NA>,<NA>,79.1,2521,6.3,swinging_strike
37,SL,Slider,2.27,6.30,7,R,L,3,2,-0.42,-0.03,-0.63,2.10,-5.836947,-120.563835,3.43,1.63,-3.071787,-3.004945,22.687948,-32.074369,2,<NA>,<NA>,<NA>,82.8,2357,6.6,swinging_strike
50,FT,2-Seam Fastball,2.38,6.35,14,R,L,1,1,1.45,0.86,1.00,2.37,-6.832388,-136.868908,3.47,1.64,-6.511650,19.794816,31.807211,-20.120004,0,<NA>,<NA>,<NA>,94.2,2317,6.5,swinging_strike
